- date: 2022-02-18 11:17:17
- author: Jerry Su
- slug: Feedback-Prize-Evaluating-Student-Writing
- title: Feedback-Pr?ize-Evaluating-Student-Writing
- category:
- tags: NLP

In [29]:
import pandas as pd
import os
import tqdm

## EDA

https://www.kaggle.com/robikscube/student-writing-competition-twitch-stream?scriptVersionId=83303421

In [37]:
df = pd.read_csv('/root/.cache/data/train.csv')

In [44]:
# Examples where the same `prediction string has two label.
df = df.loc[df[["id", "discourse_type", "predictionstring"]].duplicated(keep=False)][
    [
        "id",
        "discourse_id",
        "discourse_start",
        "discourse_end",
        "discourse_text",
        "discourse_type",
        "discourse_type_num",
        "predictionstring",
    ]
]
df

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
30713,92C09304882D,1.615603e+12,117.0,124.0,"family,",Claim,Claim 2,22
30714,92C09304882D,1.615603e+12,124.0,130.0,"money,",Claim,Claim 3,22
143325,15F434699355,1.617986e+12,201.0,210.0,knowledge,Claim,Claim 2,31
143326,15F434699355,1.617986e+12,215.0,228.0,environment.,Claim,Claim 3,31


KeyError: 0

In [31]:
df.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [32]:
df['discourse_type'].unique()

array(['Lead', 'Position', 'Evidence', 'Claim', 'Concluding Statement',
       'Counterclaim', 'Rebuttal'], dtype=object)

In [35]:
s = df['discourse_type_num'].unique()
s

array(['Lead 1', 'Position 1', 'Evidence 1', 'Evidence 2', 'Claim 1',
       'Evidence 3', 'Evidence 4', 'Claim 2', 'Evidence 5',
       'Concluding Statement 1', 'Counterclaim 1', 'Rebuttal 1',
       'Claim 3', 'Claim 4', 'Claim 5', 'Claim 6', 'Claim 7',
       'Counterclaim 2', 'Rebuttal 2', 'Counterclaim 3', 'Rebuttal 3',
       'Evidence 6', 'Lead 2', 'Counterclaim 4', 'Counterclaim 5',
       'Counterclaim 6', 'Evidence 7', 'Claim 8', 'Evidence 8',
       'Concluding Statement 2', 'Rebuttal 4', 'Rebuttal 5', 'Claim 9',
       'Position 2', 'Claim 10', 'Claim 11', 'Claim 12', 'Evidence 9',
       'Concluding Statement 3', 'Concluding Statement 4', 'Evidence 10',
       'Evidence 11', 'Rebuttal 6', 'Evidence 12'], dtype=object)

In [5]:
ids = df['id'].unique()
ids.size

15594

In [27]:
def get_instance_with_row(df, idx: int):
    row = df.loc[idx].to_dict()
    file_name = os.path.join('/root/.cache/data/train', row['id'] + '.txt')
    with open(file_name, 'r') as fp:
        text = fp.read()
    row['text'] = text
    pred_ls =  row['predictionstring'].split(' ')
    row['start_word'], row['end_word'] = int(pred_ls[0]), int(pred_ls[-1])
    row['discourse_words'] = ' '.join(text.split()[row['start_word']:row['end_word'] + 1])
    return row                                  

In [28]:
instance = get_instance_with_row(df, 1)
instance

{'id': '423A1CA112E2',
 'discourse_id': 1622627653021.0,
 'discourse_start': 230.0,
 'discourse_end': 312.0,
 'discourse_text': 'They are some really bad consequences when stuff happens when it comes to a phone.',
 'discourse_type': 'Position',
 'discourse_type_num': 'Position 1',
 'predictionstring': '45 46 47 48 49 50 51 52 53 54 55 56 57 58 59',
 'text': "Phones\n\nModern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it.\n\nWhen people have phones, they know about certain apps that they have .Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or te

In [20]:
from transformers import BigBirdTokenizerFast
tokenizer = BigBirdTokenizerFast.from_pretrained('allenai/longformer-large-4096')


def mapping_word_to_token(word_ids, word_start, word_end):
    token_start, token_end = -1, -1
    for idx, word_id in enumerate(word_ids):
        if word_id == word_start:
            token_start = idx
            break
    for idx, word_id in enumerate(word_ids):
        if word_id == word_end:
            token_end = idx
    return token_start, token_end

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
text = instance['text']

In [ ]:
ss.split()

### (discourse_start, discourse_end) 与 predictionstring时什么关系？

In [ ]:

def stat_relation(df):
    cnt = 0
    stat = {}
    err = []
    for idx, item in tqdm.tqdm(df.iterrows()):
        id_, discourse_id, start, end, discourse_text, type_, type_num, pred_str = item
        file_name = os.path.join('/root/.cache/data/train', id_ + '.txt')
        with open(file_name, 'r') as fp:
            text = fp.read()
        
        #print(discourse_text)
        # text using char.
        text_with_char = text[int(start):int(end)]
        
        if discourse_text != text_with_char:
            text_with_char = text[int(start):int(end) - 1]
            if discourse_text[:-1] == text_with_char:
                stat['char'] = stat.get('char', 0) + 1
            else:
                err.append(idx)
        else:
            stat['char'] = stat.get('char', 0) + 1
            
        cnt += 1
        if cnt == 6000000:
            break

    return stat
        

stat_relation(df)

## 统计predstring正确数

In [16]:
def stat_predstring(df):
    cnt = 0
    stat = {}
    err = []
    for idx, item in tqdm.tqdm(df.iterrows()):
        id_, discourse_id, start_, end_, discourse_text, type_, type_num, pred_str = item
        file_name = os.path.join('/root/.cache/data/train', id_ + '.txt')
        with open(file_name, 'r') as fp:
            text = fp.read()
        
        pred_ls =  pred_str.split(' ')
        start, end = int(pred_ls[0]), int(pred_ls[-1])
        text_pred = text.split()[start:end + 1]
        text_pred = ' '.join(text_pred)
        if discourse_text == text_pred:
            stat['word'] = stat.get('word', 0) + 1
        else:
            print('='*200)
            print(discourse_text)
            print()
            print(text_pred)
            print('='*200)
        cnt += 1
        if cnt == 50:
            break
    return stat
stat_predstring(df)

49it [00:00, 612.29it/s]

Some certain areas in the United States ban phones from class rooms just because of it. 

Some certain areas in the United States ban phones from class rooms just because of it.
When people have phones, they know about certain apps that they have .Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. People always have different ways how to communicate with a phone. Phones have changed due to our generation. 

When people have phones, they know about certain apps that they have .Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. People always have different ways how to communicate with a phone. Phones have changed due to our generation.
That's why there's a thing that's called no texting while driving. That's a really important thing to rememb

{'word': 13}

## char vs word vs token

In [ ]:
from transformers import BigBirdTokenizerFast

In [ ]:
tokenizer = BigBirdTokenizerFast.from_pretrained('allenai/longformer-large-4096')

In [ ]:
tokenized_inputs = tokenizer(text.split(), truncation=True, is_split_into_words=True)
tokenized_inputs

In [ ]:
len(tokenized_inputs['input_ids'])

In [ ]:
encoded_text = tokenizer.encode_plus(text.split(), add_special_tokens=False, return_offsets_mapping=True, is_split_into_words=True)
encoded_text

In [ ]:
print(tokenizer.convert_ids_to_tokens(encoded_text['input_ids']))

In [ ]:
print(text.split())

In [ ]:
cnt = 0
for word in encoded_text['offset_mapping']:
    if word[0] == 0:
        cnt += 1
cnt

In [ ]:
len(text.split())

In [ ]:
tokenizer.tokenize('PROPER_NAME')

In [ ]:
text.split().__len__()

In [ ]:
encoded_text = tokenizer.encode_plus(text, add_special_tokens=False, return_offsets_mapping=True, is_split_into_words=False)
encoded_text

In [ ]:
tt = 'Phones\n\nModern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it.\n\nWhen people have phones, they know about certain apps that they have .Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. People always have different ways how to communicate with a phone. Phones have changed due to our generation.\n\nDriving is one of the way how to get around. '

In [ ]:
tt.find('They are always on their phone')

In [ ]:
d = {}

In [ ]:
d[12] =1

In [ ]:
d

In [ ]:
d[12] = d.get(12, 0) + 1

In [ ]:
d[13] = 1

In [ ]:
d

In [ ]:
dict(sorted(d.items(), key=lambda x:x[0], reverse=True))